# Parameterized inventories

This notebook shows how to create parameterized inventories with constants, variables, and formulas.

Note that Brightway2 allows you to write real models in Python, and this is preferable to writing a model in formulas inside an inventory. However, for simple cases parameterized inventories can add real power without too much work.

We start by creating a new project and adding the basic data.

In [1]:
from brightway2 import *

In [2]:
projects.set_current("parameterized")

In [3]:
bw2setup()

Biosphere database already present!!! No setup is needed


## Create a database

This will hold our parameterized inventories.

In [4]:
db = Database("Woweee!")
db.write({})

## Create a new activity

Each activity has its own namespace for variable and formula names.

Lets also define some simple parameters.

In [5]:
act = db.new_activity("first", name="first parameterized inventory", unit="kilogram")

In [6]:
act['parameters'] = {
    'speed': {'amount': 40, 'unit': 'km/h'},
    'time': {'amount': 2.5, 'unit': 'hours', 'comment': 'A long time, but no so long'},
    'distance': {'formula': 'speed * time', 'arbitrary fields': 'can be added as needed'}
}
act.save()

## Evaluate our parameter set

Evaluate our parameter set using the [bw2parameters library](http://brightway2-parameters.readthedocs.org/).

In [7]:
from bw2parameters import ParameterSet

ps = ParameterSet(act['parameters'])
ps.evaluate()

{'distance': 100.0, 'speed': 40, 'time': 2.5}

## Add parameterized exchanges

We don't have any other technosphere processes, but we can paramerize links to biosphere flows.

In [8]:
exc = act.new_exchange(input=Database("biosphere3").random(), type='biosphere')
exc.input

'Acetic acid' (kilogram, None, ('air',))

We have a bunch of [built in functions from the Python math library and numpy](http://newville.github.io/asteval/basics.html#built-in-functions). Note that we don't define the `amount` field - this will be added later when the formula is evaluated.

In [9]:
exc['formula'] = 'sqrt(distance) * pi'

Let's do another one:

In [10]:
exc2 = act.new_exchange(input=Database("biosphere3").random(), type='biosphere')
exc2['formula'] = 'log(speed) ** 2'

The syntax to evaluate the parameterized inventories is maybe not the most intuitive, and could change in the future. We use our `ParameterSet` as a function, and call it with a list of our exchanges:

In [11]:
ps([exc, exc2])

AttributeError
   sqrt(distance) * pi
'Call' object has no attribute 'starargs'
AttributeError
   log(speed) ** 2
'Call' object has no attribute 'starargs'


[Exchange with missing fields (call ``valid(why=True)`` to see more),
 Exchange with missing fields (call ``valid(why=True)`` to see more)]

Our exchanges have been evaluated, and the `amount` field has been added. This is the basic form of a parameterized inventory.

## Global parameters

We can also have database-level parameters which can be reference by any activity, even one without it's own local parameters.

Currently we have no global parameters:

In [12]:
db.parameters

Database parameter set with 0 values

Let's add a constant, a variable, and a formula:

In [13]:
from stats_arrays import *

db.parameters['constant'] = {'amount': 42}
db.parameters['variable'] = {
    'amount': 1, 
    'loc': 1,  # Uncertainty parameters
    'minimum': 0, 
    'maximum': 3, 
    'uncertainty type': TriangularUncertainty.id
}
db.parameters['formula'] = {'formula': 'constant + variable'}

In [14]:
act2 = db.new_activity("second", name="second parameterized inventory", unit="kilogram")
act2.save()
exc3 = act2.new_exchange(input=Database("biosphere3").random(), type='biosphere')
exc3['formula'] = 'formula + 7'

When evaluating with global parameters, we still have to pass our local parameter set. Because we don't have any, we can pass an empty dictionary.

Note that we have to evaluate the global parameters.

In [15]:
ps = ParameterSet({}, global_params=db.parameters.evaluate())
ps([exc3])

[Exchange: 50 kilogram 'Cadmium' (kilogram, None, ('soil', 'industrial')) to 'second parameterized inventory' (kilogram, GLO, None)>]

This is the current state of parameterized inventories in Brightway2. One important caveat is that parameterized inventories dont work yet with Monte Carlo sampling - it is not simple to do this in an efficient way, and the methodology is still being thought about.

For anything complicated, it is (again) recommended to write a separate model in Python and use that model to dynamically generate life cycle inventories.